## 이진 검색 알고리즘  

이진검색은 내림차순이나 오름차순으로 정렬된 배열에서 좀더 효율적이다.

In [ ]:
from typing import Any, Sequence

## sequence a 에서 key와 일치하는 원소를 이진 검색
def bin_search(a : Sequence, key : Any) -> int:
    
    pl = 0              # 검색범위 맨 앞 원소의 인덱스
    pr = len(a) - 1     # 검색범위 맨 끝 원소의 인덱스

    while True:
        pc = (pl + pr) // 2     # 중앙 원소의 인덱스
        if a[pc] == key:
            return pc           # key 값이 중앙의 원소와 같으면 검색 성공
        elif a[pc] < key:       # key 값이 중앙의 원소값 보다 클 경우
            pl = pc + 1         # 검색범위를 뒤쪽 절반으로 좁힘
        else:                   # key 값이 중앙의 원소값 보다 작을 경우
            pr = pc - 1         # 검색범위를 앞쪽 절반으로 좁힘
        
        if pl > pr:             # 맨앞 원소 인덱스가 맨 끝 원소 인덱스 보다 크면 break
            break

    return -1               # 검색 실패


if __name__ == '__main__':
    num = int(input('원소 수를 입력하세요. : '))
    x = [None] * num

    print('배열 데이터를 오름차순으로 입력하세요.')

    x[0] = int(input('x[0] : '))
    for i in range(1, num):
        while True:
            x[i] = int(input(f'x[{i}]: '))
            if x[i] >= x[i - 1]:            # 직전에 입력한 숫자보다 작은수를 입력
                break

    ky = int(input('검색할 값을 입력하세요 : '))

    idx = bin_search(x, ky)

    if idx == -1:
        print('검색한 값이 존재하지 않는다.')
    else:
        print('{0}값은 x[{1}]에 있습니다.'.format(ky, idx))
    

## 해시법

해시법(hashing)은 '데이터를 저장할 위치 = 인덱스'를 간단한 연산으로 구하는 것을 말한다.  
이 방법은 원소의 검색뿐 아니라 추가, 삭제도 효율적으로 수행할 수 있다.

In [ ]:
# 체인법으로 해시함수 구현하기

from __future__ import annotations
from typing import Any, Type
import hashlib

class Node:
    # 해시를 구성하는 노드

    def __init__(self, key: Any, value: Any, next: Node) -> None:
        self.key = key
        self.value = value
        self.next = next

class ChainedHash:
    # 체인법으로 해시 클래스 구현

    def __init__(self, capacity: int) -> None:
        self.capacity = capacity        # 해시테이블의 크기를 지정
        self.table = [None] * self.capacity # 해시테이블(리스트)을 지정

    def hash_value(self, key : Any) -> int:
        #해시값을 구함
        if isinstance(key, int):
            return key % self.capacity
        return(int(hashlib.sha256(str(key).encode()).hexdigest(), 16) % self.capacity)  # key가 int형이 아닐경우    
    
    def search(self, key : Any) -> Any:
        hash = self.hash_value(key)     # 검색하는 키의 해시값
        p = self.table[hash]            # 노드를 주목

        while p is not None:
            if p.key == key:
                return p.value          # 검색 성공
            p = p.next                  # 뒤쪽 노드를 주목

        return None                     # 검색 실패
    
    def add(self, key : Any, value : Any) -> bool:
        hash = self.hash_value(key)     # 추가하는 key의 해시값
        p = self.table[hash]            # 노드를 주목

        while p is not None:
            if p.key == key:
                return False            # 추가 실패
            
            p = p.next                  # 뒤쪽 노드를 주목

        temp = Node(key, value, self.table[hash])
        self.table[hash] = temp         # 노드를 추가
        return True                     # 추가 성공
    
    def remove(self, key : Any) -> bool:
        hash = self.hash_value(key)     # 삭제할 key의 해시값
        p = self.table[hash]            # 노드를 주목
        pp = None                       # 바로 앞의 노드를 주목

        while p is not None:
            if p.key == key:            # key를 발견하면 아래를 실행
                if pp is None:
                    self.table[hash] = p.next
                else:
                    pp.next = p.next
                return True             # key 삭제 성공
            
            pp = p
            p = p.next                  # 뒤쪽 노드를 주목
        
        return False                    #삭제 실패(key가 존재하지 않음)
    
    # 해시테이블의 내용을 모두 출력한다.
    def Dump(self) -> None:
        for i in range(self.capacity):
            p = self.table[i]
            print(i, end = '')
            while p is not None:
                print(f' -> {p.key} ({p.value})', end = '')
                p = p.next
            
            print()
        

# 체인법을 구현하는 해시 클래스 ChainedHash의 사용

from enum import Enum

Menu = Enum('Menu', ['추가', '삭제', '검색', '덤프', '종료'])

def select_menu() -> Menu:
    s = [f'({m.value}){m.name}' for m in Menu]
    while True:
        print(*s, sep = '  ', end = '')
        n = int(input(': '))
        if 1 <= n <= len(Menu):
            return Menu(n)
        
hash = ChainedHash(13)

while True:
    menu = select_menu()

    if menu == Menu.추가: 
        key = int(input('추가할 키를 입력하세요 : '))
        val = input('추가할 값을 입력하세요 : ')
        if not hash.add(key, val):
            print('추가를 실패했습니다.')

    elif menu == Menu.삭제:
        key = int(input('삭제할 키를 입력하세요 : '))
        if not hash.remove(key):
            print('삭제를 실패했습니다.')
    elif menu == Menu.검색:
        key = int(input('검색할 키를 입력하세요 : '))
        t = hash.search(key)

        if t is not None:
             print(f'검색한 키를 갖는 값은 {t}입니다')
        else:
            print('검색할 데이터가 없습니다.')
    elif menu == Menu.덤프:
        hash.Dump()
    else:
        break